In [1]:
import meshio
import numpy as np

import ufl
from dolfinx import fem, io, cpp, plot
from dolfinx.mesh import create_mesh
import basix

from mpi4py import MPI
# from petsc4py.PETSc import ScalarType
# import pyvista
from petsc4py import PETSc

from gmsh_helpers import gmsh_model_to_mesh
from mesh_cylinder import mesh_cylinder

In [2]:
def create_mesh(mesh, cell_type, prune_z=False):
    cells = mesh.get_cells_type(cell_type)
    cell_data = mesh.get_cell_data("gmsh:physical", cell_type)
    points = mesh.points[:,:2] if prune_z else mesh.points
    out_mesh = meshio.Mesh(points=points, cells={cell_type: cells}, cell_data={"name_to_read":[cell_data]})
    return out_mesh

In [3]:
#It works with the msh4 only!!
msh = meshio.read("thick_cylinder.msh")

# Create and save one file for the mesh, and one file for the facets 
triangle_mesh = create_mesh(msh, "triangle", prune_z=True)
line_mesh = create_mesh(msh, "line", prune_z=True)
meshio.write("thick_cylinder.xdmf", triangle_mesh)
meshio.write("mt.xdmf", line_mesh)

with io.XDMFFile(MPI.COMM_WORLD, "thick_cylinder.xdmf", "r") as xdmf:
    mesh = xdmf.read_mesh(name="Grid")
    ct = xdmf.read_meshtags(mesh, name="Grid")

mesh.topology.create_connectivity(mesh.topology.dim, mesh.topology.dim - 1)

with io.XDMFFile(MPI.COMM_WORLD, "mt.xdmf", "r") as xdmf:
    ft = xdmf.read_meshtags(mesh, name="Grid")

In [2]:
#Geometry and mesh parameters
h = 0.03
Ri = 1.0
Re = 1.3

gmsh_model, tdim, tag_names = mesh_cylinder(Ri, Re, h, 2)

Info    : Optimizing mesh (Netgen)...
Info    : Done optimizing mesh (Wall 1.943e-06s, CPU 8e-06s)
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 18 (Circle)
Info    : [ 30%] Meshing curve 19 (Circle)
Info    : [ 50%] Meshing curve 20 (Line)
Info    : [ 80%] Meshing curve 21 (Line)
Info    : Done meshing 1D (Wall 0.000971458s, CPU 0.002082s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Delaunay)
Info    : Done meshing 2D (Wall 0.0265596s, CPU 0.026657s)
Info    : 855 nodes 1711 elements


In [3]:
# import gmsh

# # Initialise gmsh and set options
# gmsh.initialize()

In [3]:
if MPI.COMM_WORLD.rank == 0:
    # Get mesh geometry
    geometry_data = io.extract_gmsh_geometry(gmsh_model)
    # Get mesh topology for each element
    topology_data = io.extract_gmsh_topology_and_markers(gmsh_model)

In [9]:
if MPI.COMM_WORLD.rank == 0:
    # Extract the cell type and number of nodes per cell and broadcast
    # it to the other processors 
    gmsh_cell_type = list(topology_data.keys())[1]    
    properties = gmsh_model.mesh.getElementProperties(gmsh_cell_type)
    name, dim, order, num_nodes, local_coords, _ = properties
    cells = topology_data[gmsh_cell_type]["topology"]
    cell_id, num_nodes = MPI.COMM_WORLD.bcast([gmsh_cell_type, num_nodes], root=0)
else:        
    cell_id, num_nodes = MPI.COMM_WORLD.bcast([None, None], root=0)
    cells, geometry_data = np.empty([0, num_nodes]), np.empty([0, tdim])

In [10]:
list(topology_data.keys())

[1, 2]

In [11]:
# Permute topology data from MSH-ordering to dolfinx-ordering
ufl_domain = io.ufl_mesh_from_gmsh(cell_id, tdim)
gmsh_cell_perm = io.cell_perm_gmsh(cpp.mesh.to_type(str(ufl_domain.ufl_cell())), num_nodes)
cells = cells[:, gmsh_cell_perm]

# Create distributed mesh
mesh = create_mesh(MPI.COMM_WORLD, cells, geometry_data[:, :tdim], ufl_domain)

In [12]:
with io.XDMFFile(MPI.COMM_SELF, "mesh_rank_{}.xdmf".format(MPI.COMM_WORLD.rank), "w") as file:
    file.write_mesh(mesh)

In [2]:
#Geometry and mesh parameters
h = 0.03
Ri = 1.0
Re = 1.3

# Create the mesh of the specimen with given dimensions
gmsh_model, tdim, tag_names = mesh_cylinder(Ri, Re, h, 2)
# Get mesh and meshtags
mesh, mts = gmsh_model_to_mesh(gmsh_model, cell_data=False, facet_data=True, gdim=tdim)
interfaces_keys = tag_names["facets"]

Info    : Optimizing mesh (Netgen)...
Info    : Done optimizing mesh (Wall 2.027e-06s, CPU 8e-06s)
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 18 (Circle)
Info    : [ 30%] Meshing curve 19 (Circle)
Info    : [ 50%] Meshing curve 20 (Line)
Info    : [ 80%] Meshing curve 21 (Line)
Info    : Done meshing 1D (Wall 0.000918533s, CPU 0.001458s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Delaunay)
Info    : Done meshing 2D (Wall 0.0314793s, CPU 0.032076s)
Info    : 855 nodes 1711 elements


AttributeError: 'dolfinx.cpp.graph.AdjacencyList_int32' object has no attribute 'astype'

In [5]:
comm = MPI.COMM_WORLD

# Material parameters
E = 1.0
nu = 0.3
lmbda = E * nu / (1 + nu) / (1 - 2 * nu)
mu = E / 2.0 / (1 + nu)
sig0 = 5  # yield strength

#Geometry and mesh parameters
h = 0.05 / 3
Ri = 1.0
Re = 1.3

# Create the mesh of the specimen with given dimensions
gmsh_model, tdim, tag_names = mesh_cylinder(Ri, Re, h, 2)
# Get mesh and meshtags
mesh, mts = gmsh_model_to_mesh(gmsh_model, cell_data=False, facet_data=True, gdim=tdim)
interfaces_keys = tag_names["facets"]

outdir = "output_demo_plasticity"
prefix = os.path.join(outdir, "output_demo_plasticity")

if comm.rank == 0:
    Path(outdir).mkdir(parents=True, exist_ok=True)

with io.XDMFFile(comm, f"{prefix}.xdmf", "w", encoding=io.XDMFFile.Encoding.HDF5) as file:
    file.write_mesh(mesh)

deg_u = 2
deg_stress = 2

# Measures
dx = ufl.Measure(
    "dx",
    domain=mesh,
    metadata={"quadrature_degree": deg_stress, "quadrature_scheme": "default"},
)
ds = ufl.Measure("ds", domain=mesh)

#Initializing Functions and Function spaces
# element_u = ufl.VectorElement("CG", mesh.ufl_cell(), degree=deg_u, dim=tdim)
# V_u = fem.FunctionSpace(mesh, element_u)
# V_uxy = fem.FunctionSpace(
#     mesh, ufl.FiniteElement("CG", mesh.ufl_cell(), degree=deg_u)
# )

# W_element = ufl.VectorElement(
#     "Quadrature", mesh.ufl_cell(), deg_stress, quad_scheme="default", dim=4
# )
# W = fem.FunctionSpace(mesh, W_element)

# W0_element = ufl.FiniteElement(
#     "Quadrature", mesh.ufl_cell(), deg_stress, quad_scheme="default"
# )
# W0 = fem.FunctionSpace(mesh, W0_element)

# W0_plot_element = ufl.FiniteElement("DG", mesh.ufl_cell(), degree=deg_stress)
# W0_plot = fem.FunctionSpace(mesh, W0_plot_element)
deg_u = 2
deg_stress = 2
V = fem.VectorFunctionSpace(mesh, ("CG", deg_u))
We = ufl.VectorElement("Quadrature", mesh.ufl_cell(), degree=deg_stress, dim=4, quad_scheme='default')
W0e = ufl.FiniteElement("Quadrature", mesh.ufl_cell(), degree=deg_stress, quad_scheme='default')
W = fem.FunctionSpace(mesh, We)
W0 = fem.FunctionSpace(mesh, W0e)

sig = fem.Function(W)
sig_old = fem.Function(W)
n_elas = fem.Function(W)

beta = fem.Function(W0)
p = fem.Function(W0)
dp = fem.Function(W0)
# p_plot = fem.Function(W0_plot, name="Cumulative plastic strain")
# dp_plot = fem.Function(W0_plot)

u = fem.Function(V)
du = fem.Function(V)
Du = fem.Function(V)

zero_u = fem.Function(V)
zero_Du = fem.Function(V)
loading = fem.Function(
    fem.FunctionSpace(
        mesh, ufl.FiniteElement("CG", mesh.ufl_cell(), degree=deg_u)
    )
)

v = ufl.TrialFunction(V)
u_ = ufl.TestFunction(V)

#Setting Bcs
# dofs_u_bottom = fem.locate_dofs_geometrical(
#     (V_u.sub(1), V_uxy), lambda x: np.isclose(x[1], 0)
# )
# dofs_u_left = fem.locate_dofs_geometrical(
#     (V_u.sub(0), V_uxy), lambda x: np.isclose(x[0], 0.0)
# )

# bcs_u = [
#     fem.dirichletbc(zero_u, dofs_u_bottom, V_u.sub(1)),
#     fem.dirichletbc(zero_u, dofs_u_left, V_u.sub(0)),
# ]

left_marker = 3
down_marker = 1
left_facets = ft.indices[ft.values == left_marker]
down_facets = ft.indices[ft.values == down_marker]
left_dofs = fem.locate_dofs_topological(V.sub(0), mesh.topology.dim-1, left_facets)
down_dofs = fem.locate_dofs_topological(V.sub(1), mesh.topology.dim-1, down_facets)

bcs = [fem.dirichletbc(ScalarType(0), left_dofs, V.sub(0)), fem.dirichletbc(ScalarType(0), down_dofs, V.sub(1))]

n = ufl.FacetNormal(mesh)
q_lim = 2.0 / ufl.sqrt(3) * ufl.ln(Re / Ri) * sig0

loading.interpolate(lambda x: (np.zeros_like(x[1])))
loading.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)


def F_ext(v):
    return -loading * ufl.dot(n, v) * ds(4)


def eps(v):
    e = ufl.sym(ufl.grad(v))
    return ufl.as_tensor([[e[0, 0], e[0, 1], 0], [e[0, 1], e[1, 1], 0], [0, 0, 0]])


def sigma(eps_el):
    return lmbda * ufl.tr(eps_el) * ufl.Identity(3) + 2 * mu * eps_el


def as_3D_tensor(X):
    return ufl.as_tensor([[X[0], X[3], 0], [X[3], X[1], 0], [0, 0, X[2]]])


def ppos(x):
    return (x + ufl.sqrt(x**2)) / 2.0


def proj_sig(deps, old_sig):
    sig_n = as_3D_tensor(old_sig)
    sig_elas = sig_n + sigma(deps)
    s = ufl.dev(sig_elas)
    sig_eq = ufl.sqrt(3 / 2.0 * ufl.inner(s, s))
    f_elas = sig_eq - sig0
    dp = ppos(f_elas) / (3 * mu)
    n_elas = s / sig_eq * ppos(f_elas) / f_elas
    beta = 3 * mu * dp / sig_eq
    new_sig = sig_elas - beta * s
    return (
        ufl.as_vector([new_sig[0, 0], new_sig[1, 1], new_sig[2, 2], new_sig[0, 1]]),
        ufl.as_vector([n_elas[0, 0], n_elas[1, 1], n_elas[2, 2], n_elas[0, 1]]),
        beta,
        dp,
    )


def sigma_tang(e):
    N_elas = as_3D_tensor(n_elas)
    return (
        sigma(e)
        - 3 * mu * (1 - beta) * ufl.inner(N_elas, e) * N_elas
        - 2 * mu * beta * ufl.dev(e)
    )

In [6]:
quadrature_points, wts = basix.make_quadrature(basix.CellType.triangle, deg_stress)
map_c = mesh.topology.index_map(mesh.topology.dim)
num_cells = map_c.size_local + map_c.num_ghosts
cells = np.arange(0, num_cells, dtype=np.int32)

a_Newton = ufl.inner(eps(v), sigma_tang(eps(u_))) * dx
res = -ufl.inner(eps(u_), as_3D_tensor(sig)) * dx + F_ext(u_)
problem = fem.petsc.LinearProblem(
    a_Newton,
    res,
    bcs=bcs,
    petsc_options={
        "pc_type": "lu",
        "pc_factor_mat_solver_type": "mumps",
    },
)

In [8]:
q_lim

1.5147607871604416

In [15]:
Nitermax, tol = 200, 1e-8  # parameters of the Newton-Raphson procedure
Nincr = 20
load_steps = np.linspace(0.0, 1.1, Nincr + 1)[1:] ** 0.5
# for (i, t) in enumerate(load_steps):
i = 0
t = load_steps[0]
loading.interpolate(lambda x: (t * q_lim * np.ones_like(x[1])))
loading.vector.ghostUpdate(
    addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD
)
print(Res.array.max())
nRes = 1.0
zero_Du.vector.copy(Du.vector)
Du.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
print("Increment:", str(i + 1), t * q_lim)
niter = 0
while nRes > tol and niter < Nitermax:
    du = problem.solve()
    Du.vector.axpy(1, du.vector)
    Du.vector.ghostUpdate(
        addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD
    )
    print(Du.x.array.max())
    print(Du.x.array.min())
    deps = eps(Du)

    Res = fem.petsc.assemble_vector(fem.form(res))
    Res.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
    fem.set_bc(Res, bcs, du.vector)
    nRes = ufl.sqrt(Res.dot(Res))
    print("    Residual:", nRes, dp.vector.max()[1])
    niter += 1

0.0
Increment: 1 0.355242893422994
0.0
0.0
    Residual: 0.0 0.0


In [16]:
def project(v, target_func, bcs=[]):
    # v->target_func
    # Ensure we have a mesh and attach to measure
    V = target_func.function_space
    # dx = ufl.dx(V.mesh)

    # Define variational problem for projection
    w = ufl.TestFunction(V)
    Pv = ufl.TrialFunction(V)
    a = fem.form(ufl.inner(Pv, w) * dx)
    L = fem.form(ufl.inner(v, w) * dx)

    # Assemble linear system
    A = fem.petsc.assemble_matrix(a, bcs)
    A.assemble()
    b = fem.petsc.assemble_vector(L)
    fem.petsc.apply_lifting(b, [a], [bcs])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
    fem.petsc.set_bc(b, bcs)

    # Solve linear system
    solver = PETSc.KSP().create(A.getComm())
    solver.setOperators(A)
    solver.solve(b, target_func.vector)

In [8]:
Nitermax, tol = 200, 1e-8  # parameters of the Newton-Raphson procedure
Nincr = 20
load_steps = np.linspace(0.0, 1.1, Nincr + 1)[1:] ** 0.5
for (i, t) in enumerate(load_steps):
    loading.interpolate(lambda x: (t * q_lim * np.ones_like(x[1])))
    loading.vector.ghostUpdate(
        addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD
    )
    nRes = 1.0
    zero_Du.vector.copy(Du.vector)
    Du.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    print("Increment:", str(i + 1), t * q_lim)
    niter = 0
    while nRes > tol and niter < Nitermax:
        du = problem.solve()
        Du.vector.axpy(1, du.vector)
        Du.vector.ghostUpdate(
            addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD
        )
        print(Du.x.array.max())
        print(Du.x.array.min())
        deps = eps(Du)

        sig_, n_elas_, beta_, dp_ = proj_sig(deps, sig_old)
       
        # project(sig_, sig)
        # project(n_elas_, n_elas)
        # project(beta_, beta)
        sig_expr = fem.Expression(sig_, quadrature_points)
        sig_eval = sig_expr.eval(cells)
        with sig.vector.localForm() as sig_local:
            sig_local.setBlockSize(sig.function_space.dofmap.bs)
            sig_local.setValuesBlocked(
                W.dofmap.list.array, sig_eval, addv=PETSc.InsertMode.INSERT
            )

        n_elas_expr = fem.Expression(n_elas_, quadrature_points)
        n_elas_eval = n_elas_expr.eval(cells)
        with n_elas.vector.localForm() as n_elas_local:
            n_elas_local.setBlockSize(n_elas.function_space.dofmap.bs)
            n_elas_local.setValuesBlocked(
                W.dofmap.list.array, n_elas_eval, addv=PETSc.InsertMode.INSERT
            )

        beta_expr = fem.Expression(beta_, quadrature_points)
        beta_eval = beta_expr.eval(cells)
        with beta.vector.localForm() as beta_local:
            beta_local.setBlockSize(beta.function_space.dofmap.bs)
            beta_local.setValuesBlocked(
                W0.dofmap.list.array, beta_eval, addv=PETSc.InsertMode.INSERT
            )

        Res = fem.petsc.assemble_vector(fem.form(res))
        Res.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
        fem.set_bc(Res, bcs, du.vector)
        nRes = ufl.sqrt(Res.dot(Res))
        print("    Residual:", nRes, dp.vector.max()[1])
        niter += 1
        
    u.vector.axpy(1, Du.vector)
    u.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    sig.vector.copy(sig_old.vector)
    sig_old.vector.ghostUpdate(
        addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD
    )

    dp_expr = fem.Expression(dp_, quadrature_points)
    dp_eval = dp_expr.eval(cells)
    with dp.vector.localForm() as dp_local:
        dp_local.setBlockSize(dp.function_space.dofmap.bs)
        dp_local.setValuesBlocked(
            W0.dofmap.list.array, dp_eval, addv=PETSc.InsertMode.INSERT
        )
    p.vector.axpy(1, dp.vector)
    p.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
    # dp_expr_plot = fem.Expression(dp_, quadrature_points)
    # dp_plot.interpolate(dp_expr_plot)
    # p_plot.vector.axpy(1, dp_plot.vector)
    # p_plot.vector.ghostUpdate(
    #     addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD
    # )

    # with XDMFFile(comm, f"{prefix}.xdmf", "a", encoding=XDMFFile.Encoding.HDF5) as file:
    #     file.write_function(u, t)
    #     file.write_function(p_plot, t)

Increment: 1 0.355242893422994
0.0
0.0
    Residual: nan nan
Increment: 2 0.502389317815458
inf
inf
    Residual: nan nan
Increment: 3 0.6152987404364013
inf
inf
    Residual: nan nan
Increment: 4 0.710485786845988
inf
inf
    Residual: nan nan
Increment: 5 0.7943472582175275
inf
inf
    Residual: nan nan
Increment: 6 0.8701638236362416
inf
inf
    Residual: nan nan
Increment: 7 0.9398843510202649
inf
inf
    Residual: nan nan
Increment: 8 1.004778635630916
inf
inf
    Residual: nan nan
Increment: 9 1.0657286802689818
inf
inf
    Residual: nan nan
Increment: 10 1.1233766658051103
inf
inf
    Residual: nan nan
Increment: 11 1.178207386924283
inf
inf
    Residual: nan nan
Increment: 12 1.2305974808728026
inf
inf
    Residual: nan nan
Increment: 13 1.2808464674807938
inf
inf
    Residual: nan nan
Increment: 14 1.3291971962750933
inf
inf
    Residual: nan nan
Increment: 15 1.375849810085792
inf
inf
    Residual: nan nan
Increment: 16 1.420971573691976
inf
inf
    Residual: nan nan
Incremen